## Imports

In [103]:
import numpy as np
import pandas as pd 
import os as os
from time import time, strftime
from datetime import datetime

from sklearn.pipeline import Pipeline, FeatureUnion, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import accuracy_score, explained_variance_score, r2_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

import warnings
warnings.filterwarnings('ignore')

## Create Classes for Data Processing

In [81]:
class Load_Data(BaseEstimator, TransformerMixin):
    def __init__(self, features=None):
        self.features = features
        self.weather_dir = ''
        self.soil_dir = ''
        self.drop_columns = ['STATION', 'NAME', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 'AWND_ATTRIBUTES', 'PGTM_ATTRIBUTES', 
                             'PSUN', 'PSUN_ATTRIBUTES', 'SNOW', 'SNOW_ATTRIBUTES', 'SNWD', 'SNWD_ATTRIBUTES', 'TAVG',
                             'TAVG_ATTRIBUTES', 'TMAX_ATTRIBUTES', 'TMIN_ATTRIBUTES', 'TSUN', 'TSUN_ATTRIBUTES', 'WDF2_ATTRIBUTES', 
                             'WDF5_ATTRIBUTES', 'WSF2_ATTRIBUTES','WSF5_ATTRIBUTES', 'WT01_ATTRIBUTES', 'WT02_ATTRIBUTES', 
                             'WT03_ATTRIBUTES', 'WT06_ATTRIBUTES', 'WT08_ATTRIBUTES', 'PRCP_ATTRIBUTES']
        
    def fit(self, w_dir, s_dir):
        self.weather_dir = w_dir
        self.soil_dir = s_dir
        return self
    
    def transform(self, X):
        #Aggregate all 43 files into one file
        file_list = os.listdir(self.soil_dir)
        agg_data = pd.DataFrame()
        for file in file_list:
            path = self.soil_dir + file
            curr_data = pd.read_csv(path, sep='\t')
            agg_data = agg_data.append(curr_data)
        
        #Drop rows with only NAs for measurement values
        soil = agg_data.dropna(thresh=10)
        
        #Import weather files and drop unnessecary fields
        weather = pd.read_csv(self.weather_dir)
        drop_cols = list(set(weather.columns).intersection(self.drop_columns))
        weather = weather.drop(columns = self.drop_columns)
        
        #Convert both files to use same datetime
        soil['Date'] = pd.to_datetime(soil['Date'])
        weather['DATE'] = pd.to_datetime(weather['DATE'])
        
        #Join previous 16 days weather to moisture readings
        for i in range(0, 17):
            weather_new = weather.add_suffix('_' + str(i))
            soil = soil.merge(weather_new, how = 'left', left_on = 'Date', right_on = weather['DATE'] - pd.DateOffset(i * -1))
            
        #Store the month of the reading as a feature
        soil['Month'] = pd.DatetimeIndex(soil['Date']).month
        
        date_attribs = ['Date', 'DATE_0', 'DATE_1', 'DATE_2', 'DATE_3', 'DATE_4','DATE_5', 'DATE_6', 'DATE_7', 'DATE_8', 'DATE_9', 'DATE_10', \
                        'DATE_11', 'DATE_12', 'DATE_13', 'DATE_14', 'DATE_15', 'DATE_16']
        
        if 'DATE_0' in list(soil.columns):
            soil.drop(columns = date_attribs, inplace = True)
        soil['Location'] = soil['Location'].astype('object')
            
        return soil

In [82]:
class Feature_Engineer(BaseEstimator, TransformerMixin):
    def __init__(self, features=None):
        self.features = features
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        #Add categorical feature that simply stores if it rained that day or not
        for i in range(17):
            col_name = 'PRCP_' + str(i)
            rain_y_n_name = 'RAIN_Y_N_' + str(i)
            X[rain_y_n_name] = np.nan
            X[rain_y_n_name].loc[X[col_name] > 0] = 1
            X[rain_y_n_name].loc[X[col_name] == 0] = 0
            X[rain_y_n_name] = X[rain_y_n_name].astype('object')
        return X

In [83]:
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        print(X)
        return X[self.attribute_names].values

In [84]:
class Convert_Date(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names = None):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X['Date'] = pd.to_timedelta(X['Date']).dt.total_seconds().astype(int)
        return X

## Create a Pipeline That Uses Data Processing Class

In [85]:
%%time
soil_file_dir = '../data/soil/'
weather_file_dir = '../data/weather/weather_data.csv'
x = 0

pre_work_pipeline = Pipeline([
    ('prework', Load_Data()),
    ('features', Feature_Engineer())
])

pre_work_pipeline.fit(weather_file_dir, soil_file_dir)
prework_df = pre_work_pipeline.transform(x)
#Save to CSV so that we do not need to import and clean data everytime
prework_df.to_csv('clean_data.csv')

Wall time: 22.6 s


## Make Classifier Label

In [86]:
y_cols = ['VW_30cm', 'VW_60cm', 'VW_90cm', 'VW_120cm', 'VW_150cm']

for cols in y_cols:
    name = cols[3:] + '_class'
    prework_df[name] = ''
    prework_df[name].loc[(prework_df[cols] <= 0.1)] = '0.1'
    prework_df[name].loc[(prework_df[cols] > 0.1) & (prework_df[cols] <= 0.2)] = '0.2'
    prework_df[name].loc[(prework_df[cols] > 0.2) & (prework_df[cols] <= 0.3)] = '0.3'
    prework_df[name].loc[(prework_df[cols] > 0.3) & (prework_df[cols] <= 0.4)] = '0.4'
    prework_df[name].loc[(prework_df[cols] > 0.4) & (prework_df[cols] <= 0.5)] = '0.5'
    prework_df[name].loc[(prework_df[cols] > 0.5) & (prework_df[cols] <= 0.6)] = '0.6'
    prework_df[name].loc[(prework_df[cols] > 0.6) & (prework_df[cols] <= 0.7)] = '0.7'
    prework_df[name].loc[(prework_df[cols] > 0.7) & (prework_df[cols] <= 0.8)] = '0.8'
    prework_df[name].loc[(prework_df[cols] > 0.8)] = '0.9'

## Make Data Frames for Each Depth

The moisture data is taken at various depths. We want to build models seperately for different depths. So we need to make a dataframe for each depth so that we can elminate entire rows where the predictor is NA


In [87]:
# First split out y values
all_y_cols = ['VW_30cm', 'VW_60cm', 'VW_90cm', 'VW_120cm', 'VW_150cm', '30cm_class', '60cm_class', '90cm_class', '120cm_class', '150cm_class']
X_sets = {}
y_sets = {}
x_cols = [col for col in prework_df.columns if col not in y_cols]
X = prework_df.loc[:, x_cols]
#y = prework_df.loc[:, y_cols]

for cols in all_y_cols:
    if cols[:1] == 'V':
        dataset_name = cols[3:]
    else:
        dataset_name = cols
    holder = prework_df.dropna(subset = [cols])
    X_sets[dataset_name] = holder[x_cols].fillna(0)
    y_sets[dataset_name] = holder[cols]

## Split Train and Test

In [88]:
# Split training and test data
# 80-20 ratio
# Trying to keep same ratios for each location using stratify
# Could have done this in the cell above, but wanted a seperate step for this
X_train_set = {}
X_test_set = {}
y_train_set = {}
y_test_set = {}

for cols in all_y_cols:
    if cols[:1] == 'V':
        dataset_name = cols[3:]
    else:
        dataset_name = cols 
    X_train_set[dataset_name], X_test_set[dataset_name], y_train_set[dataset_name], y_test_set[dataset_name] = train_test_split(X_sets[dataset_name], y_sets[dataset_name], \
                                                                                                                                test_size=0.2, stratify = X_sets[dataset_name]['Location'], random_state=42)

## Generic Pipeline

In [106]:
num_attribs = X_train_set['60cm'].select_dtypes(exclude=['object', 'category']).columns
cat_attribs = X_train_set['60cm'].select_dtypes(include=['object', 'category']).columns

numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value = 0)),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value = '')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, num_attribs),
        ('cat', categorical_transformer, cat_attribs)
    ])

pipe_with_estimator = Pipeline(steps=[('preprocessor', preprocessor),
                                      ('classifier', RandomForestRegressor())])



## Linear Regression Tests

In [618]:
pipe_with_estimator.fit(X_train_set['60cm'], y_train_set['60cm'])


Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  Pipeline(memory=None,
                                                           steps=[('imputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                                 copy=True,
                                                                                 fill_value=0,
                                                                                 missing_values=nan,
                                                                                 strategy='constant',
                                                              

In [622]:
explained_variance_score(y_test_set['60cm'], pipe_with_estimator.predict(X_test_set['60cm']))

0.9013707703385528

In [640]:
data_cols = ['30cm', '60cm', '90cm', '120cm', '150cm']
try:
    log
except NameError:
    log = pd.DataFrame(columns = ['Experiment', 'Depth', 'Fit_Time', 'Pred_Time', 'r2_score', 'exp_var_score', 'datetime'])
    
for cols in data_cols:
    t0 = time()
    pipe_with_estimator.fit(X_train_set[cols], y_train_set[cols])
    t1 = time()
    preds = pipe_with_estimator.predict(X_test_set[cols])
    t2 = time()
    expvar = explained_variance_score(y_test_set[cols], preds)
    r2sc = r2_score(y_test_set[cols], preds)
    now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    log.loc[len(log)] = ['First Linear Reg', cols, t1-t0, t2-t1, r2sc, expvar, now]
    
print(log)

         Experiment  Depth   Fit_Time  Pred_Time  r2_score  exp_var_score  \
0  First Linear Reg   30cm  58.701572   0.238179  0.890208       0.890208   
1  First Linear Reg   60cm  58.847758   0.169551  0.898520       0.898522   
2  First Linear Reg   90cm  54.927837   0.172255  0.882179       0.882180   
3  First Linear Reg  120cm  64.877650   0.197685  0.884033       0.884033   
4  First Linear Reg  150cm  63.256277   0.170247  0.876900       0.876920   

              datetime  
0  2020-11-02 20:10:26  
1  2020-11-02 20:11:25  
2  2020-11-02 20:12:20  
3  2020-11-02 20:13:25  
4  2020-11-02 20:14:28  


## Trying Different Classifiers

In [115]:
# Adapted from notebook from INFO526 SP2020
def ConductGridSearch(X_train, y_train, X_test, y_test, i=0, prefix='', n_jobs=-1, verbose=1):    
    classifiers = [
        ('Logistic Regression', LogisticRegression(random_state=42, max_iter=4000, multi_class = 'multinomial', solver = 'newton-cg')),
        #('K-Nearest Neighbors', KNeighborsClassifier()),
        #('Support Vector', SVC(random_state=42)),
        #('Stochastic GD', SGDClassifier(random_state=42)),
        #('RandomForest', RandomForestClassifier()),
    ]

    # Arrange grid search parameters for each classifier
    params_grid = {
        'Logistic Regression': {
            'penalty': ('l2', 'none'),
            'tol': (0.0001, 0.00001, 0.0000001), 
            'C': (10, 1, 0.1, 0.01),
        },
        'K-Nearest Neighbors': {
            'n_neighbors': (3, 5, 7, 8, 11),
            'p': (1,2),
        },
        #'Naive Bayes': {},
        'Support Vector' : {
            'kernel': ('rbf', 'poly'),     
            'degree': (1, 2, 3, 4, 5),
            'C': (10, 1, 0.1, 0.01),
        },
        'Stochastic GD': {
            'loss': ('hinge', 'perceptron', 'log'),
            'penalty': ('l1', 'l2', 'elasticnet'),
            'tol': (0.0001, 0.00001, 0.0000001), 
            'alpha': (0.1, 0.01, 0.001, 0.0001), 
        },
        'RandomForest':  {
            'max_depth': [9, 15, 22, 26, 30],
            'max_features': [1, 3, 5],
            'min_samples_split': [5, 10, 15],
            'min_samples_leaf': [3, 5, 10],
            'bootstrap': [False],
            'n_estimators':[20, 80, 150, 200, 300]},
    }
    
    for (name, classifier) in classifiers:
        i += 1
        # Print classifier and parameters
        print('****** START',prefix, name,'*****')
        parameters = params_grid[name]
        print("Parameters:")
        for p in sorted(parameters.keys()):
            print("\t"+str(p)+": "+ str(parameters[p]))
        
        # generate the pipeline
        full_pipeline_with_predictor = Pipeline([
        ("preparation", preprocessor),
        ("predictor", classifier)
        ])
        
        # Execute the grid search
        params = {}
        for p in parameters.keys():
            pipe_key = 'predictor__'+str(p)
            params[pipe_key] = parameters[p] 
        grid_search = GridSearchCV(full_pipeline_with_predictor, params, scoring='roc_auc', cv=5, 
                                   n_jobs=n_jobs, verbose=verbose)
        grid_search.fit(X_train, y_train)
                
        # Best estimator score
        best_train = pct(grid_search.best_score_)

        # Best estimator fitting time
        start = time()
        grid_search.best_estimator_.fit(x_train, y_train)
        train_time = round(time() - start, 4)

        # Best estimator prediction time
        start = time()
        best_test_accuracy = pct(grid_search.best_estimator_.score(X_test, y_test))
        test_time = round(time() - start, 4)

        best_train_scores = cross_val_score(grid_search.best_estimator_, x_train, y_train, cv=cv30Splits)
        best_train_accuracy = pct(best_train_scores.mean())
 
        (t_stat, p_value) = stats.ttest_rel(logit_scores, best_train_scores)
        
        # Collect the best parameters found by the grid search
        print("Best Parameters:")
        best_parameters = grid_search.best_estimator_.get_params()
        param_dump = []
        for param_name in sorted(params.keys()):
            param_dump.append((param_name, best_parameters[param_name]))
            print("\t"+str(param_name)+": " + str(best_parameters[param_name]))
        print("****** FINISH",prefix,name," *****")
        print("")
        
        # Record the results
        results.loc[i] = [prefix+name, best_train_accuracy, best_test_accuracy, round(p_value,5), train_time, test_time, json.dumps(param_dump)]

In [ ]:
# Still trying to get this to work
curr = '30cm_class'
ConductGridSearch(X_train_set[curr], y_train_set[curr], X_test_set[curr], y_test_set[curr], 0, "Best Model:",  n_jobs=-1,verbose=1)